In [2]:
%matplotlib inline
import importlib
from pytorch_utils_oh_2 import *

Pytorch utils oh: pytorch_utils_oh_2.py
Pytorch: 0.2.0_4


In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
import pytorch_utils_oh_2; importlib.reload(pytorch_utils_oh_2); from pytorch_utils_oh_2 import *;

Pytorch utils oh: pytorch_utils_oh_2.py
Pytorch: 0.2.0_4


# Data loading

In [29]:
all_data = pickle.load(open("data/en_train_fixed_1.pkl", "rb" ))
all_data_sentence_index = all_data.set_index('sentence_id')

In [4]:
categories_all = sorted(all_data["class"].unique())
print(categories_all)
print(len(categories_all))
categories_index = dict((c, i) for i, c in enumerate(categories_all))

['ELECTRONIC', 'LETTERS', 'NOT_CHANGED', 'NUMBERS', 'PLAIN', 'VERBATIM']
6


### After words handling

In [5]:
words_after_common = pickle.load(open("data/en_features/words_after_all.pkl", "rb"))
words_after_common = [EOS_TOKEN, SOS_TOKEN, UNKNOWN_WORD_TOKEN, NUMBER_WORD_TOKEN, SAMPLE_WORD_TOKEN] + words_after_common
words_after_common[0:7]
words_after_index = dict((c, i) for i, c in enumerate(words_after_common))

['<EOS>', '<SOS>', '<UNK>', '<0000>', '<SAMPLE>', 'two', 'twenty']

In [102]:
pickle.dump(words_after_common, open("data/en_features/words_after_ext.pkl", 'wb'))

In [103]:
len(words_after_index)

1351

In [6]:
words_after_by_length = sorted(words_after_common, key=len, reverse=True)
words_after_regex = re.compile('(' + ')|('.join(words_after_by_length) + ')')

In [7]:
def after_sentence_to_word_indexes(sentence, include_eos=True):
    reg = re.finditer(words_after_regex, sentence)
    arr = [words_after_index[s[0]] for s in reg]
    if include_eos:
        arr += [words_after_index[EOS_TOKEN]]
    return arr
tmp = after_sentence_to_word_indexes('one plus two equals one hundred')
tmp
[words_after_common[s] for s in tmp]

[9, 1212, 5, 1252, 9, 10, 0]

['one', 'plus', 'two', 'equals', 'one', 'hundred', '<EOS>']

# Preparing data a_word_ind

In [11]:
changed_data = all_data[all_data['class'] != 'NOT_CHANGED']
len(changed_data)

659544

In [13]:
len(changed_data[changed_data['after'].str.len() == changed_data['before'].str.len()])
len(changed_data[changed_data['after'].str.len() != changed_data['before'].str.len()])
len(changed_data[changed_data['after'].str.len() > changed_data['before'].str.len()])
len(changed_data[changed_data['after'].str.len() < changed_data['before'].str.len()])

9527

650017

600384

49633

In [14]:
changed_data[0:1]

,sentence_id,token_id,class,before,after,class_org
10,1,0,NUMBERS,2006,two thousand six,DATE


In [17]:
def prepare_after_by_rows(pd_rows):
    global current_row
    iter_len = len(pd_rows)
    rows_iter = pd_rows.itertuples()
    iteration_idx = 0
    start = time.time()
    
    for sample_row in log_progress(rows_iter, every=10, size=iter_len):
        before = sample_row.before
        current_row = sample_row

        a_word_ind = after_sentence_to_word_indexes(sample_row.after, include_eos=True)
        
        all_data.at[sample_row.Index, 'a_word_ind'] = a_word_ind

        iteration_idx += 1
        if iteration_idx%10000 == 0:
            print("{:>7d} {:>2.2%} ({:>8})".format(iteration_idx, iteration_idx/iter_len, time_since(start)))

In [23]:
all_data.sample(2)

,sentence_id,token_id,class,before,after,class_org,a_word_ind
3753960,288855,12,NOT_CHANGED,roughly,roughly,PLAIN,NaN
9800619,739388,4,NOT_CHANGED,author,author,PLAIN,NaN


In [40]:
all_data.at[0, 'a_word_ind'] = [1, 12]
#all_data.head()
all_data.at[0, 'a_word_ind'] = all_data.at[1, 'a_word_ind']
#all_data.head()

In [50]:
prepare_after_by_rows(changed_data)

  10000 1.52% (  0m 24s)
  20000 3.03% (  0m 49s)
  30000 4.55% (  1m 14s)
  40000 6.06% (  1m 40s)
  50000 7.58% (   2m 5s)
  60000 9.10% (  2m 30s)
  70000 10.61% (  2m 55s)
  80000 12.13% (  3m 20s)
  90000 13.65% (  3m 46s)
 100000 15.16% (  4m 12s)
 110000 16.68% (  4m 37s)
 120000 18.19% (   5m 3s)
 130000 19.71% (  5m 29s)
 140000 21.23% (  5m 54s)
 150000 22.74% (  6m 19s)
 160000 24.26% (  6m 45s)
 170000 25.78% (  7m 11s)
 180000 27.29% (  7m 37s)
 190000 28.81% (   8m 2s)
 200000 30.32% (  8m 26s)
 210000 31.84% (  8m 51s)
 220000 33.36% (  9m 17s)
 230000 34.87% (  9m 43s)
 240000 36.39% (  10m 7s)
 250000 37.90% ( 10m 34s)
 260000 39.42% ( 10m 59s)
 270000 40.94% ( 11m 25s)
 280000 42.45% ( 11m 51s)
 290000 43.97% ( 12m 18s)
 300000 45.49% ( 12m 43s)
 310000 47.00% (  13m 9s)
 320000 48.52% ( 13m 34s)
 330000 50.03% ( 13m 59s)
 340000 51.55% ( 14m 25s)
 350000 53.07% ( 14m 50s)
 360000 54.58% ( 15m 15s)
 370000 56.10% ( 15m 40s)
 380000 57.62% (  16m 5s)
 390000 59.13% ( 1

In [57]:
all_data[all_data['class'] != 'NOT_CHANGED'].sample(10)

,sentence_id,token_id,class,before,after,class_org,a_word_ind
393639,31595,0,LETTERS,U.S.,u s,LETTERS,"[43, 17, 0]"
4627569,354438,4,LETTERS,pp,p p,LETTERS,"[24, 24, 0]"
3875346,297935,1,PLAIN,organises,organizes,PLAIN,"[364, 0]"
8659494,654956,8,NUMBERS,II,two,CARDINAL,"[5, 0]"
6733858,511669,13,LETTERS,PDF,p d f,LETTERS,"[24, 26, 37, 0]"
3608707,277851,17,NUMBERS,2/2013,two two thousand thirteenths,FRACTION,"[5, 5, 8, 373, 0]"
3348284,258230,7,NUMBERS,857,eight hundred fifty seven,CARDINAL,"[16, 10, 38, 18, 0]"
9157563,691631,10,NUMBERS,1st,first,ORDINAL,"[56, 0]"
856597,68192,7,NUMBERS,12325,twelve thousand three hundred twenty five,CARDINAL,"[47, 8, 13, 10, 6, 14, 0]"
6268151,477020,13,NUMBERS,"July 12, 1991",july twelfth nineteen ninety one,DATE,"[67, 95, 7, 23, 9, 0]"


In [54]:
pickle.dump(all_data, open("data/en_train_fixed_2_a_word_ind.pkl", "wb" ))

# Preparing data sentence_tokenized

In [56]:
all_data = pickle.load(open("data/en_train_fixed_2_a_word_ind.pkl", "rb" ))
all_data_sentence_index = all_data.set_index('sentence_id')

In [58]:
changed_data = all_data[all_data['class'] != 'NOT_CHANGED']

In [59]:
changed_data.sample(2)

,sentence_id,token_id,class,before,after,class_org,a_word_ind
664310,53038,14,NUMBERS,2009,two thousand nine,DATE,"[5, 8, 15, 0]"
6496403,493979,8,NUMBERS,1929,nineteen twenty nine,DATE,"[7, 6, 15, 0]"


In [88]:
def prepare_sentence_tokenized_by_rows(pd_rows):
    global current_row
    iter_len = len(pd_rows)
    rows_iter = pd_rows.itertuples()
    iteration_idx = 0
    start = time.time()
    
    for sample_row in log_progress(rows_iter, every=10, size=iter_len):
        before = sample_row.before
        current_row = sample_row

        sent_rows = all_data_sentence_index.loc[sample_row.sentence_id]
        befores = list(sent_rows.before)
        token_id_idx = list(sent_rows['token_id']).index(sample_row.token_id)
        
        befores = [simple_tokeniser(w) for w in befores]
        befores[token_id_idx] = [SAMPLE_WORD_TOKEN]
        befores = np.concatenate(befores)
    
        sentence = ' '.join(befores)

        all_data.at[sample_row.Index, 'sentence'] = sentence
        
        iteration_idx += 1
        if iteration_idx%50000 == 0 or iteration_idx == 5000:
            print("{:>7d} {:>2.2%} ({:>8})".format(iteration_idx, iteration_idx/iter_len, time_since(start)))

In [85]:
prepare_sentence_tokenized_by_rows(changed_data)

  10000 1.52% (   0m 7s)
  20000 3.03% (  0m 14s)
  30000 4.55% (  0m 21s)
  40000 6.06% (  0m 28s)
  50000 7.58% (  0m 35s)
  60000 9.10% (  0m 42s)
  70000 10.61% (  0m 49s)
  80000 12.13% (  0m 56s)
  90000 13.65% (   1m 4s)
 100000 15.16% (  1m 12s)
 110000 16.68% (  1m 19s)
 120000 18.19% (  1m 27s)
 130000 19.71% (  1m 35s)
 140000 21.23% (  1m 42s)
 150000 22.74% (  1m 50s)
 160000 24.26% (  1m 58s)
 170000 25.78% (   2m 5s)
 180000 27.29% (  2m 13s)
 190000 28.81% (  2m 21s)
 200000 30.32% (  2m 28s)
 210000 31.84% (  2m 36s)
 220000 33.36% (  2m 44s)
 230000 34.87% (  2m 52s)
 240000 36.39% (  2m 59s)
 250000 37.90% (   3m 7s)
 260000 39.42% (  3m 14s)
 270000 40.94% (  3m 22s)
 280000 42.45% (  3m 30s)
 290000 43.97% (  3m 37s)
 300000 45.49% (  3m 45s)
 310000 47.00% (  3m 52s)
 320000 48.52% (   4m 0s)
 330000 50.03% (   4m 8s)
 340000 51.55% (  4m 16s)
 350000 53.07% (  4m 24s)
 360000 54.58% (  4m 32s)
 370000 56.10% (  4m 40s)
 380000 57.62% (  4m 48s)
 390000 59.13% (  

In [86]:
all_data[9:14]

,sentence_id,token_id,class,before,after,class_org,a_word_ind,sentence
9,0,9,NOT_CHANGED,.,.,PUNCT,NaN,NaN
10,1,0,NUMBERS,2006,two thousand six,DATE,"[5, 8, 20, 0]",<SAMPLE> iucn red list of threatened species .
11,1,1,LETTERS,IUCN,i u c n,LETTERS,"[31, 43, 21, 29, 0]",2006 <SAMPLE> red list of threatened species .
12,1,2,NOT_CHANGED,Red,Red,PLAIN,NaN,NaN
13,1,3,NOT_CHANGED,List,List,PLAIN,NaN,NaN


In [87]:
pickle.dump(all_data, open("data/en_train_fixed_3_changed_sentences.pkl", "wb" ))

In [89]:
prepare_sentence_tokenized_by_rows(all_data)

   5000 0.05% (   0m 3s)
  50000 0.50% (  0m 35s)
 100000 1.01% (  1m 11s)
 150000 1.51% (  1m 47s)
 200000 2.02% (  2m 24s)
 250000 2.52% (  2m 59s)
 300000 3.02% (  3m 35s)
 350000 3.53% (  4m 11s)
 400000 4.03% (  4m 47s)
 450000 4.54% (  5m 22s)
 500000 5.04% (  5m 58s)
 550000 5.55% (  6m 34s)
 600000 6.05% (  7m 10s)
 650000 6.55% (  7m 46s)
 700000 7.06% (  8m 23s)
 750000 7.56% (  8m 59s)
 800000 8.07% (  9m 35s)
 850000 8.57% ( 10m 11s)
 900000 9.07% ( 10m 47s)
 950000 9.58% ( 11m 23s)
1000000 10.08% (  12m 0s)
1050000 10.59% ( 12m 35s)
1100000 11.09% ( 13m 12s)
1150000 11.59% ( 13m 49s)
1200000 12.10% ( 14m 25s)
1250000 12.60% (  15m 1s)
1300000 13.11% ( 15m 39s)
1350000 13.61% ( 16m 14s)
1400000 14.12% ( 16m 50s)
1450000 14.62% ( 17m 28s)
1500000 15.12% (  18m 5s)
1550000 15.63% ( 18m 42s)
1600000 16.13% ( 19m 19s)
1650000 16.64% ( 19m 56s)
1700000 17.14% ( 20m 33s)
1750000 17.64% (  21m 8s)
1800000 18.15% ( 21m 44s)
1850000 18.65% ( 22m 19s)
1900000 19.16% ( 22m 54s)
195000

In [90]:
all_data[9:14]

,sentence_id,token_id,class,before,after,class_org,a_word_ind,sentence
9,0,9,NOT_CHANGED,.,.,PUNCT,NaN,brillantaisia is a genus of plant in family ac...
10,1,0,NUMBERS,2006,two thousand six,DATE,"[5, 8, 20, 0]",<SAMPLE> iucn red list of threatened species .
11,1,1,LETTERS,IUCN,i u c n,LETTERS,"[31, 43, 21, 29, 0]",2006 <SAMPLE> red list of threatened species .
12,1,2,NOT_CHANGED,Red,Red,PLAIN,NaN,2006 iucn <SAMPLE> list of threatened species .
13,1,3,NOT_CHANGED,List,List,PLAIN,NaN,2006 iucn red <SAMPLE> of threatened species .


In [91]:
pickle.dump(all_data, open("data/en_train_fixed_4_sentences.pkl", "wb" ))

In [105]:
len(changed_data[changed_data['after'].str.len() > 100])

867

In [96]:
changed_data.loc[8875393]['before']

'college.usc.edu/unruh/private/UnruhNews.pdfhttp://atticwritersworkshop.com/teacher/peter-zuckermanhttp://www.falconartcommunity.com/artists.htmlhttp://www.oregonlive.com/mapes/index.ssf/2014/05/gay_marriage_supporters_launch.htmlhttp://www.advocate.com/oregon/2014/05/02/watch-oregons-first-ad-defeat-turn-away-gay-initiativehttp://www.oregonlive.com/mapes/index.ssf/2014/05/oregon_gay_marriage_ban_struck.htmlhttp://www.oregonlive.com/mapes/index.ssf/2014/05/gay_marriage_backers_of_exempt.htmlhttp://www.newapproachoregon.com/contact/http://www.bendbulletin.com/localstate/2283636-151/oregon-pot-campaign-raises-over-1-millionhttp://hemp.org/news/content/oregon-addiction-expert-kicks-23-million-marijuana-legalization-ad-campaignhttp://newsok.com/legal-pot-is-top-oregon-news-story-of-2014/article/feed/777847http://www.oregonlive.com/mapes/index.ssf/2014/08/sponsors_cancel_drug_summit_in.html#incart_riverhttp://www.oregonlive.com/politics/index.ssf/2014/11/recreational_marijuana_passes.htmlht

# Manually checked removing

In [5]:
all_data = pickle.load(open("data/en_train_fixed_4_sentences.pkl", "rb" ))

In [6]:
chars_normal, chars_normal_index = load_characters_pkl('data/en_features/chars_normal.pkl')
common_words, common_words_index = load_common_words_10k()
chars_with_changes = pickle.load(open('data/en_features/chars_with_changes.pkl', "rb"))
chars_with_no_changes_re = re.compile("[^{}]".format(''.join(chars_with_changes)))
chars_with_single_output_dict = pickle.load(open('data/en_features/chars_with_single_output_dict.pkl', "rb"));

In [7]:
def manual_pre_checks(before):
    if chars_with_no_changes_re.search(before): #contains chars that were never changed
        return before 
    if before in chars_with_single_output_dict:
        return chars_with_single_output_dict[before]
    return False

In [41]:
manual_pre_checks('κ')

'kappa'

In [9]:
changed_data = all_data[all_data['class'] != 'NOT_CHANGED']

In [12]:
changed_data['class'].unique()

array(['NUMBERS', 'LETTERS', 'PLAIN', 'VERBATIM', 'ELECTRONIC'], dtype=object)

In [29]:
def prepare_manual_class_by_rows(pd_rows):
    global current_row
    iter_len = len(pd_rows)
    rows_iter = pd_rows.itertuples()
    iteration_idx = 0
    start = time.time()
    
    for sample_row in log_progress(rows_iter, every=1000, size=iter_len):
        before = sample_row.before
        current_row = sample_row

        result = manual_pre_checks(sample_row.before)
        if result:
            if result != sample_row.after:
                print("PROBLEM with:", sample_row)
            else:
                all_data.at[sample_row.Index, 'class'] = 'MANUAL'

        iteration_idx += 1
        if iteration_idx%50000 == 0:
            print("{:>7d} {:>2.2%} ({:>8})".format(iteration_idx, iteration_idx/iter_len, time_since(start)))

In [30]:
prepare_manual_class_by_rows(changed_data)

  50000 7.58% (   0m 0s)
 100000 15.16% (   0m 0s)
 150000 22.74% (   0m 0s)
 200000 30.32% (   0m 0s)
 250000 37.90% (   0m 0s)
 300000 45.49% (   0m 0s)
 350000 53.07% (   0m 0s)
 400000 60.65% (   0m 1s)
 450000 68.23% (   0m 1s)
 500000 75.81% (   0m 1s)
 550000 83.39% (   0m 1s)
 600000 90.97% (   0m 1s)
 650000 98.55% (   0m 1s)


In [33]:
changed_data[changed_data['class'] == 'MANUAL']

,sentence_id,token_id,class,before,after,class_org,a_word_ind,sentence


In [38]:
all_data[all_data['class'] == 'MANUAL'].sample(5)

,sentence_id,token_id,class,before,after,class_org,a_word_ind,sentence
5033346,384879,29,MANUAL,ο,omicron,VERBATIM,"[135, 0]",great synaxaristes : ( greek ) ἡ ἁ γ ι α ἄ ν ν...
8241234,623993,21,MANUAL,κ,kappa,VERBATIM,"[167, 0]",124 basic training wing ( 124 π τ ε ρ υ γ α β ...
7089991,538096,16,MANUAL,ν,nu,VERBATIM,"[152, 0]","in greek mythology , pleisthenes ( ancient gre..."
360513,28983,9,MANUAL,θ,theta,VERBATIM,"[237, 0]","melantho ( greek : μ ε λ α ν <SAMPLE> ω ) , is..."
373892,30049,18,MANUAL,δ,delta,VERBATIM,"[208, 0]",thomas garaklidis ( greek : θ ω μ α ς γ κ α ρ ...


In [39]:
pickle.dump(all_data, open("data/en_train_fixed_5_manual.pkl", "wb" ))